In [7]:
import pandas as pd
import numpy as np

In [14]:
normalized_data = pd.read_csv("normalized_stock_data.csv")

print(f"there are {len(normalized_data)} rows")
normalized_data.head()

there are 262 rows


,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0.002198,0.000487,0.006397,0.001871,0.000000,0.4,0.0
1,0.001843,0.000000,0.003596,0.001100,0.269540,0.0,0.0
2,0.000939,0.002022,0.005530,0.003165,0.277210,0.0,0.0
3,0.003480,0.003948,0.008077,0.005852,0.386136,0.0,0.0
4,0.006016,0.004188,0.010104,0.005416,0.132201,0.0,0.0


In [15]:
# create segmented sequences
def create_sequence(data, seq_len):
    sequences = []
    for i in range(len(data) - seq_len):
        sequences.append(data[i : i + seq_len])
    return np.array(sequences)

sequence_length = 3
sequences = create_sequence(normalized_data, sequence_length)
# [[row1, row2, row3], .... []]

print(len(sequences))

259


In [19]:
# train-test split (70-30 split)
split_ratio = 0.7
train_size = int(len(sequences) * split_ratio)

print(f"train size: {train_size}")
print(f"test size: {len(sequences) - train_size}")

train_sequences = sequences[:train_size]
test_sequences = sequences[train_size:]

print(f"train sequences: {len(train_sequences)}")
print(f"test sequences: {len(test_sequences)}")

train size: 181
test size: 78
train sequences: 181
test sequences: 78
